In [30]:
from torchvision import datasets
import torch
data_folder = 'CARD 20'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

In [31]:
val_fmnist = datasets.FashionMNIST(data_folder, download=True, train=False)
val_images = val_fmnist.data
val_targets = val_fmnist.targets

In [32]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [33]:
# modelo com duas camadas
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float()
        x = x.view(-1, 28*28)/255
        self.x, self.y = x.to(device), y.to(device)

    def __getitem__(self, ix):
        x = self.x[ix]
        y = self.y[ix]
        return x.to(device), y.to(device)

    # tive que adicionar a função len pois não estava funcionando, e ai percebi que não olhei direito
    def __len__(self):
        return len(self.x)

In [34]:
def get_model():
    model = nn.Sequential(
        nn.Linear(28*28, 1000), # 28*28 conectado a 1000 neuronios
        nn.ReLU(),
        nn.Linear(1000, 100), # 1000 neuronios também conectado a 100
        nn.ReLU(),
        nn.Linear(1000, 10) # 1000 conectado a 10
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [35]:
# função de treino
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

# função de validação do modelo
def accuracy(x, y, model):
    model.eval()

    with torch.no_grad():
        prediction = model(x)

    max_values, argmaxes = prediction.max(-1)  # obtém as classes previstas
    is_correct = argmaxes == y  # verifica se a previsão está correta
    return is_correct.cpu().numpy().tolist()  # retorna a lista de acertos


In [36]:
@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [37]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)

    val = FMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=False)
    return trn_dl, val_dl

In [38]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(5):
    print(f'epoch atual: {epoch}')

    train_epoch_losses, train_epoch_accuracies = [], []

    # loop pelos batches do conjunto de treino
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss) # armazena a perda do batch

    train_epoch_loss = np.array(train_epoch_losses).mean() # média da perda da época atual

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model) # calcula a precisao
        validation_loss = val_loss(x, y, model) # calcula a perda de validação

    # média da precisão de validação
    val_epoch_accuracy = np.mean(val_is_correct)

    # adiciona a perda no treinamento na lista
    train_losses.append(train_epoch_loss)

    # adiciona a precisão de treino na lista
    train_accuracies.append(train_epoch_accuracies)

    # adiciona a perda de validação na lista
    val_losses.append(validation_loss)

    # adiciona a precisão de validação na lista
    val_accuracies.append(val_epoch_accuracy)